In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-cde7d8ad-9aef-3a15-2e76-765d16c19f3c)


In [ ]:
!git clone https://github.com/verashira/TSPNet.git

Cloning into 'TSPNet'...
remote: Enumerating objects: 386, done.
remote: Counting objects: 100% (386/386), done.
remote: Compressing objects: 100% (302/302), done.
remote: Total 386 (delta 92), reused 365 (delta 76), pack-reused 0
Receiving objects: 100% (386/386), 1.62 MiB | 26.70 MiB/s, done.
Resolving deltas: 100% (92/92), done.


In [ ]:
%cd TSPNet
!mkdir i3d-features
!mkdir data-bin

/content/TSPNet


In [ ]:
import shutil
shutil.unpack_archive("/content/drive/MyDrive/TSPNet/my-output/span=8_stride=2.zip","/content/TSPNet/i3d-features")
shutil.unpack_archive("/content/drive/MyDrive/TSPNet/my-output/span=12_stride=2.zip","/content/TSPNet/i3d-features")
shutil.unpack_archive("/content/drive/MyDrive/TSPNet/my-output/span=16_stride=2.zip","/content/TSPNet/i3d-features")

In [ ]:
import shutil
shutil.unpack_archive("/content/drive/MyDrive/TSPNet/my-output/translation-texts/data-bin.zip","/content/TSPNet/data-bin")

In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 37.7MB/s 


![](https://drive.google.com/uc?export=view&id=1ccJ5_AV9y-ABkmzZr0vfF9o0kS97KS0V) 

###TRAIN

**Before Training: If you get error about "can not cast to long" - Replace torch.div() by torch.floor_divide() in file search.py inside fairseq folder (line 81)** 

**add the following two arguments inside train script(sh)**

'--eval-bleu-print-samples', action='store_true', help='print sample generations during validation')

'--eval-bleu-save', default=None, type=str, help='path to save the hypothesis translations.')

In [ ]:
#!/bin/bash

set -x -e -u -o pipefail
cd ..

if [ -z ${SAVE_DIR+x} ]; then 
    echo "no save dir"; 
    PARAM_SAVE_DIR=""
else 
    echo "save dir"
    PARAM_SAVE_DIR="--save-dir ${SAVE_DIR} --maximize-best-checkpoint-metric"
fi

PATIENCE=8
DROPOUT=0.4
ENC_LAYERS=1
WARMUP=0

VOCAB=25000
CUDA_VISIBLE_DEVICES=0 python train.py data-bin \
        --task translation_sign \
        --target-lang de \
        --max-tokens 4096 \
        --num-levels 3 \
        --multilv-args '{"span_lengths": [8, 12, 16], "level_links": [[1, 0], [2, 1], [2, 0]], "stride": 2, "eye": true, "same_level_links": true, "symmetric": true}' \
        --src-lv0-body-feat-root i3d-features/span=8_stride=2 \
        --src-lv1-body-feat-root i3d-features/span=12_stride=2 \
        --src-lv2-body-feat-root i3d-features/span=16_stride=2 \
        --arch transformer_sign \
        --encoder-embed-dim 1024 \
        --decoder-embed-dim 300 \
        --warmup-updates 0 \
        --lr 1e-04 \
        --lr-shrink 0.5 \
        --lr-patience ${PATIENCE} \
        --lr-scheduler reduce_lr_on_plateau \
        --lr-mode max \
        --optimizer adam \
        --activation-fn gelu \
        --criterion label_smoothed_cross_entropy \
        --valid-subset valid \
        --label-smoothing 0.1 \
        --weight-decay 0.0001 \
        --dropout ${DROPOUT} \
        --max-epoch 200 \
        --save-interval 1 \
        --eval-bleu \
        --eval-bleu-args '{"beam": 10, "max_len_a": 1.2, "max_len_b": 10}' \
        --eval-tokenized-bleu \
        --eval-bleu-remove-bpe sentencepiece \
        --eval-bleu-detok space \
        --eval-bleu-print-samples \
        --eval-bleu-save output \
        --best-checkpoint-metric bleu \
        --maximize-best-checkpoint-metric \
        --encoder-learned-pos \
        --decoder-learned-pos \
        --decoder-embed-path data-bin/emb \
        --decoder-attention-heads 10 \
        --validate-interval 1 \
        --encoder-layers ${ENC_LAYERS} \
        --no-last-checkpoints \
        --no-epoch-checkpoints \
        ${PARAM_SAVE_DIR}



In [ ]:
%cd /content/TSPNet
!mkdir output

/content/TSPNet


In [ ]:
%cd /content/TSPNet/
!pip install --editable .

/content/TSPNet
Obtaining file:///content/TSPNet
     |████████████████████████████████| 61kB 8.3MB/s 
  Running setup.py develop for fairseq


In [ ]:
%cd '/content/TSPNet/run_scripts'
!SAVE_DIR='/content/output' bash run_phoenix_pos_embed_sp_test_3lvl.sh

Streaming output truncated to the last 5000 lines.
/content/TSPNet/fairseq/sequence_generator.py:378: UserWarning: An output with one or more elements was resized since it had shape [82], which does not match the required output shape [8].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  /pytorch/aten/src/ATen/native/Resize.cpp:23.)
  out=eos_bbsz_idx,
/content/TSPNet/fairseq/sequence_generator.py:386: UserWarning: An output with one or more elements was resized since it had shape [82], which does not match the required output shape [8].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  /pytorch/

In [ ]:
%cp -R /content/output/checkpoint_best.pt /content/drive/MyDrive/TSPNet/

###TEST

In [ ]:
import shutil
shutil.copy("/content/drive/MyDrive/TSPNet/checkpoint_best.pt","/content/")

'/content/checkpoint_best.pt'

In [ ]:
#!/bin/bash

set -x -e -u -o pipefail
CHECKPOINT=$(realpath ${CHECKPOINT})

cd ..

DROPOUT=0.4
PATIENCE=8
ENC_LAYERS=1

VOCAB=25000
python test_scripts/test_sign_local.py data-bin \
        --task translation_sign \
        --target-lang de \
        --max-tokens 4096 \
        --num-levels 3 \
        --multilv-args '{"span_lengths": [8, 12, 16], "level_links": [[1, 0], [2, 1], [2, 0]], "stride": 2, "eye": true, "same_level_links": true, "symmetric": true}' \
        --src-lv0-body-feat-root i3d-features/span=8_stride=2 \
        --src-lv1-body-feat-root i3d-features/span=12_stride=2 \
        --src-lv2-body-feat-root i3d-features/span=16_stride=2 \
        --arch transformer_sign \
        --encoder-embed-dim 1024 \
        --decoder-embed-dim 300 \
        --warmup-updates 0 \
        --lr 1e-04 \
        --lr-shrink 0.5 \
        --lr-patience ${PATIENCE} \
        --lr-scheduler reduce_lr_on_plateau \
        --lr-mode max \
        --optimizer adam \
        --activation-fn gelu \
        --criterion label_smoothed_cross_entropy \
        --valid-subset test \
        --label-smoothing 0.1 \
        --weight-decay 0.0001 \
        --dropout ${DROPOUT} \
        --max-epoch 200 \
        --save-interval 200 \
        --eval-bleu \
        --eval-bleu-args '{"beam": 10, "max_len_a": 1.2, "max_len_b": 10}' \
        --eval-tokenized-bleu \
        --eval-bleu-remove-bpe sentencepiece \
        --eval-bleu-detok space \
        --eval-bleu-print-samples \
        --eval-bleu-save output \
        --best-checkpoint-metric bleu \
        --maximize-best-checkpoint-metric \
        --encoder-learned-pos \
        --decoder-learned-pos \
        --decoder-embed-path data-bin/emb \
        --decoder-attention-heads 10 \
        --encoder-layers ${ENC_LAYERS} \
        --restore-file ${CHECKPOINT}


**change div to floor_divide**

In [ ]:
%cd '/content/TSPNet/test_scripts'
!CHECKPOINT='/content/checkpoint_best.pt' bash test_phoenix_pos_embed_sp_test_3lvl.sh

/content/TSPNet/test_scripts
++ realpath /content/checkpoint_best.pt
+ CHECKPOINT=/content/checkpoint_best.pt
+ cd ..
+ DROPOUT=0.4
+ PATIENCE=8
+ ENC_LAYERS=1
+ VOCAB=25000
+ python test_scripts/test_sign_local.py data-bin --task translation_sign --target-lang de --max-tokens 4096 --num-levels 3 --multilv-args '{"span_lengths": [8, 12, 16], "level_links": [[1, 0], [2, 1], [2, 0]], "stride": 2, "eye": true, "same_level_links": true, "symmetric": true}' --src-lv0-body-feat-root i3d-features/span=8_stride=2 --src-lv1-body-feat-root i3d-features/span=12_stride=2 --src-lv2-body-feat-root i3d-features/span=16_stride=2 --arch transformer_sign --encoder-embed-dim 1024 --decoder-embed-dim 300 --warmup-updates 0 --lr 1e-04 --lr-shrink 0.5 --lr-patience 8 --lr-scheduler reduce_lr_on_plateau --lr-mode max --optimizer adam --activation-fn gelu --criterion label_smoothed_cross_entropy --valid-subset test --label-smoothing 0.1 --weight-decay 0.0001 --dropout 0.4 --max-epoch 200 --save-interval 200 -

**Quantitative Result**

![](https://drive.google.com/uc?export=view&id=16pK9ETqgTW9tMgT0-9n5jB6wWCvcrcf4) 

**Qualitative Result**

**ground truth:** 

im süden bläst der wind mäßig im norden frisch bis stark mit stürmischen böen an der see und auf den bergen schwere sturmböen in den gipfellagen von harz und erzgebirge auch orkanböen .

(in the south the wind blows moderately in the north fresh to strong with stormy gusts on the sea and on the mountains heavy gusts of wind in the peaks of Harz and Erzgebirge also gusts of gale force.)

**example hypothesis:**

im süden schwacher wind im norden weht er mäßig bis frisch mit starken böen an der see und auf den bergen stürmischen böen .

(in the south, weak winds in the north it blows moderately to fresh with strong gusts at the sea and stormy gusts on the mountains.)



**ground truth:** 

am freitag in der südosthälfte zum teil noch freundlich während sich die wolken von nordwesten verdichten und gebietsweise regen bringen .

(on Friday in the south-east half still friendly while the clouds from the north-west thicken and bring rain in some areas.)

**example hypothesis:** 

die aussichten am freitag liegen die wolken unterschiedlich dicht beieinander vor allem im süden gebietsweise regnet es .

(the views on friday the clouds are differently close together, especially in the south it rains.)



![](https://drive.google.com/uc?export=view&id=1MNI5Fy1OfL5D_S9h02rKjFX2540bf83V)

 


###POSSIBLE IMPROVEMENT PATHS:




1.   Different CNN Architecture? 
      * TSPNet used I3D.   
2.   Try To Modify the Pipeline as a Sign-to-Gloss-to-Text approach? (Introduce Gloss)
      * TSPNet followed Sign-to-Text approach.
3.   Use Keypoint Estimation and GCN? 
      * TSPNet used framewise convolution.
4.   Pipelines of Other Papers.

We are currently trying to implement the third path, with the help of another paper- **Skeleton Aware Multi-modal Sign Language Recognition.**




**Skeleton Aware Multi-modal Sign Language Recognition**

![](https://drive.google.com/uc?export=view&id=1BUh27b-DImHyejbEoJlj2fQHfBKQaYGs)
